In [50]:
import pyotp, requests, json

login_creds = {
    'user_id' : "",
    'password' : "!",
    "twofa_value" : '630190'
}

session = requests.Session() 

res1 = session.post(
    'https://kite.zerodha.com/api/login',
    data={
        "user_id": login_creds['user_id'], 
        "password": login_creds['password'], 
        "type": "user_id"}
) 

loginRes = res1.json() 
#loginRes
finalRes = session.post(
    'https://kite.zerodha.com/api/twofa',
    data={
        "user_id": login_creds['user_id'], 
        "request_id": loginRes['data']['request_id'], 
        "twofa_value": login_creds['twofa_value'],
        "twofa_type": 'app_code'
    }
) 

#finalRes.json() 

session_info = session.cookies.get_dict()

headers = {
    "Content-Type" : "application/x-www-form-urlencoded",
    "cookie": f"kf_session={session_info['kf_session']}; __cf_bm={session_info['__cf_bm']}; _cfuvid={session_info['_cfuvid']}; public_token={session_info['public_token']}",
    "Authorization": f"enctoken {session_info['enctoken']}"
}

In [ ]:
# holdings

holdings = requests.request("GET",
                             "https://kite.zerodha.com/oms/portfolio/holdings", 
                             headers=headers)
holdings.json()

In [ ]:
# positions

positions = requests.request("GET",
                 "https://kite.zerodha.com/oms/portfolio/positions", 
                 headers=headers)
positions.json()

In [ ]:
# orders

orders = requests.request("GET",
                 "https://kite.zerodha.com/oms/orders", 
                 headers=headers)
orders.json()

In [ ]:
# margins

margins = requests.request("GET",
                 "https://kite.zerodha.com/oms/user/margins", 
                 headers=headers)
margins.json()

In [ ]:
# place buy order

order_details = {
    "variety": "regular", 
    "exchange": "NSE", 
    "tradingsymbol": "IEX", 
    "transaction_type": "BUY", 
    "order_type": "MARKET", 
    "quantity": 1, 
    "price": 221, 
    "product": "CNC", 
    "validity": "DAY", 
    "disclosed_quantity": 0, 
    "trigger price": 0, 
    "squareoff": 0, 
    "stoploss": 0, 
    "trailing stoploss": 0, 
    "user_id": login_creds['user_id']
}

place_order_response = requests.request("POST",
                                 "https://kite.zerodha.com/oms/orders/regular", 
                                 headers = headers,
                                 data = order_details)
place_order_response.json()

In [23]:
# Get historic data for a stock between 2 dates

historic_data_response = requests.request("GET",
                                 f"https://kite.zerodha.com/oms/instruments/historical/969473/day?user_id={login_creds['user_id']}&oi=1&from=2023-10-02&to=2024-10-02", 
                                 headers = headers)
historic_data_response.json()

{'status': 'success',
 'data': {'candles': [['2023-10-03T00:00:00+0530',
    406.45,
    408.1,
    403.6,
    405.45,
    2864052,
    0],
   ['2023-10-04T00:00:00+0530', 402.7, 405.45, 400.65, 404.9, 3452935, 0],
   ['2023-10-05T00:00:00+0530', 407.95, 410.4, 405, 406.8, 4062652, 0],
   ['2023-10-06T00:00:00+0530', 408, 409.8, 406.95, 407.85, 3182502, 0],
   ['2023-10-09T00:00:00+0530', 404.2, 411, 403.4, 405.6, 3015225, 0],
   ['2023-10-10T00:00:00+0530', 406.4, 408.75, 406.2, 407.75, 2564007, 0],
   ['2023-10-11T00:00:00+0530', 409, 423.95, 409, 421.15, 9546114, 0],
   ['2023-10-12T00:00:00+0530', 421.75, 421.9, 414.7, 417.1, 5117199, 0],
   ['2023-10-13T00:00:00+0530', 411.9, 414.4, 409.05, 411.05, 5457027, 0],
   ['2023-10-16T00:00:00+0530', 411, 411.9, 408.15, 410.2, 2638826, 0],
   ['2023-10-17T00:00:00+0530', 411.7, 413.85, 411.05, 411.4, 2989694, 0],
   ['2023-10-18T00:00:00+0530', 409, 413.65, 407, 407.45, 2440533, 0],
   ['2023-10-19T00:00:00+0530', 391.95, 396.75, 390.1, 3

In [20]:
# Get events for a stock between 2 dates

events_response = requests.request("GET",
                                 "https://kite.zerodha.com/oms/instruments/events/345089?from=2023-07-19&to=2024-10-02", 
                                 headers = headers)
events_response.json()

{'status': 'success',
 'data': [{'action_type': 'dividend',
   'ex_date': '2024-08-01',
   'meta': {'ratio_1': 2000, 'ratio_2': 40}},
  {'action_type': 'dividend',
   'ex_date': '2024-02-21',
   'meta': {'ratio_1': 3750, 'ratio_2': 75}},
  {'action_type': 'dividend',
   'ex_date': '2023-07-27',
   'meta': {'ratio_1': 1750, 'ratio_2': 35}},
  {'action_type': 'dividend',
   'ex_date': '2024-02-21',
   'meta': {'ratio_1': 1250, 'ratio_2': 25}}]}

In [51]:
# create GTT

gtt_details = {
    'condition' : json.dumps(
        { 
            'exchange' : 'NSE',
            'tradingsymbol': 'M&M',
            'trigger_values' :[3007.2,3323.75],
            'last_price' :3165.5
        }
    ),
    'expires_at' : '2025-10-02 00:00:00',
    'orders' : json.dumps(
        [
            {
                'exchange' : 'NSE',
                'tradingsymbol': 'M&M',
                'transaction_type' : 'SELL',
                'quantity':1,
                'price' :3007.2,
                'order_type': 'LIMIT',
                'product' : 'CNC'
            },
            {
    
                'exchange' : 'NSE',
                'tradingsymbol': 'M&M',
                'transaction_type' : 'SELL',
                'quantity':1,
                'price' :3323.75,
                'order_type': 'LIMIT',
                'product' : 'CNC'
            }
        ]
    ),
    'type' : 'two-leg'
}

gtt_response = requests.request("POST",
                                 "https://kite.zerodha.com/oms/gtt/triggers", 
                                 headers = headers,
                                 data = gtt_details)
gtt_response.json()

{'status': 'success', 'data': {'trigger_id': 241422709}}

In [52]:
# current GTTS

current_gtts = requests.request("GET",
                 "https://kite.zerodha.com/oms/gtt/triggers", 
                 headers=headers)
current_gtts.json()

{'status': 'success',
 'data': [{'id': 241422709,
   'user_id': 'QOZ373',
   'parent_trigger': None,
   'type': 'two-leg',
   'created_at': '2024-10-02 11:38:06',
   'updated_at': '2024-10-02 11:38:06',
   'expires_at': '2025-10-02 11:38:06',
   'status': 'active',
   'condition': {'exchange': 'NSE',
    'last_price': 3165.5,
    'tradingsymbol': 'M&M',
    'trigger_values': [3007.2, 3323.75],
    'instrument_token': 519937},
   'orders': [{'exchange': 'NSE',
     'tradingsymbol': 'M&M',
     'product': 'CNC',
     'order_type': 'LIMIT',
     'transaction_type': 'SELL',
     'quantity': 1,
     'price': 3007.2,
     'result': None},
    {'exchange': 'NSE',
     'tradingsymbol': 'M&M',
     'product': 'CNC',
     'order_type': 'LIMIT',
     'transaction_type': 'SELL',
     'quantity': 1,
     'price': 3323.75,
     'result': None}],
   'meta': {}}]}

In [53]:
# delete GTT

delete_gtt = requests.request("DELETE",
                 "https://kite.zerodha.com/oms/gtt/triggers/241422709", 
                 headers=headers)
delete_gtt.json()

{'status': 'success', 'data': {'trigger_id': 241422709}}